In [0]:
import yaml

with open('ingestion_config.yml', 'r') as file:
    config = yaml.safe_load(file)

databricks_config = config['databricks_config']
llm_config = config['llm_config']

catalog=databricks_config['catalog']
schema=databricks_config['schema']

volume_path = databricks_config['volume_path']
bronze_table=databricks_config['bronze_table']
silver_table=databricks_config['silver_table']

In [0]:
sec_documents_raw = spark.read.format("binaryFile").load(volume_path)
print("Total docs: " + str(sec_documents_raw.count()))
display(sec_documents_raw.limit(2))

#write docs to bronze table
sec_documents_raw.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.{bronze_table}")
print(f"Successfully wrote bronze docs to {catalog}.{schema}.{bronze_table}")

In [0]:
#extract document year and company name
sec_document_year_and_company = sec_documents_raw.selectExpr("*","ai_extract(path, array('year', 'company')) AS extracted_data")

In [0]:
#extract document type
sec_document_document_type = sec_document_year_and_company.selectExpr(
    "*",
    """
    ai_classify(path, ARRAY('10k', '8k', '10q', 'Earnings Report')) AS document_type
    """,
).select("*", "extracted_data.*").drop("extracted_data")

In [0]:
sec_document_document_type.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.{silver_table}")
print(f"Successfully wrote bronze docs to {catalog}.{schema}.{silver_table}")

In [0]:
display(spark.table(f"{catalog}.{schema}.{silver_table}"))